In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
# Import pyMC3 and also arviz for visualisation
import pymc as pm
import arviz as az
# Import the other core data science packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
from sklearn import datasets
from scipy.stats import norm
import statsmodels.formula.api as smf

RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
import seaborn as sns
import os

## Data

In [ ]:
# True parameter values
alpha_, sigma_ = 1, 0.05
beta_ = [1]

# Size of dataset
size = 20

data = pd.DataFrame()
# Predictor variable
data['x1'] = np.linspace(0,1,size)

# Simulate outcome variable
data['y'] = alpha_ + beta_[0] * data['x1']
data['y_measure'] = data['y'] + rng.normal(size=size) * sigma_
data['outlier'] = False

# Adding one outlier:
index = data.index[int(size*0.60)]
data.loc[index,'outlier'] = True
data.loc[index,'y_measure']*=1.5

In [ ]:
fig,ax=plt.subplots()
data.plot(x='x1', y='y', style='r-', ax=ax)

data_good = data.loc[~data['outlier']]
data_outliers = data.loc[data['outlier']]

data_good.plot(x='x1', y='y_measure', style='.', ax=ax)
data_outliers.plot(x='x1', y='y_measure', style='.', label='outlier', ax=ax)


## OLS regression

In [ ]:
# OLS line
formula = 'y_measure ~ x1'
results_robust = smf.ols(formula, data=data_good).fit()

In [ ]:
results_robust.summary()

In [ ]:
results_ols = smf.ols(formula, data=data).fit()
results_ols.summary()

In [ ]:
X = data[['x1']]
prediction = X.copy()

prediction['robust'] = results_robust.predict(X)
prediction['OLS'] = results_ols.predict(X)

fig,ax=plt.subplots(sharex=True)
data_good.plot(x='x1', y='y_measure', style='.', ax=ax)
data_outliers.plot(x='x1', y='y_measure', label='outliers', style='.', ax=ax)
data.plot(x='x1', y='y', style='r-', ax=ax)

prediction.plot(x='x1', y='robust', style='--', ax=ax)
prediction.plot(x='x1', y='OLS', style='--', ax=ax)

## Bayesian

In [ ]:
basic_model = pm.Model()

with basic_model:
    # Priors for unknown model parameters
    alpha = pm.Normal("alpha", mu=alpha_, sigma=0.1)
    beta = pm.Normal("beta", mu=beta_, sigma=0.1, shape=1)
    sigma = pm.HalfNormal("sigma", sigma=sigma_)

    # Expected value of outcome
    mu = alpha + beta[0] * data['x1']

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=data['y_measure'])

In [ ]:
os.name

In [ ]:
import os
if os.name == 'nt':
    os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'

pm.model_to_graphviz(basic_model)

In [ ]:
with basic_model:
    # draw 1000 posterior samples
    idata = pm.sample(draws=100)

In [ ]:
with basic_model:
    az.plot_posterior(idata,
                      var_names=['alpha', 'beta', 'sigma'],
                      textsize=18,
                      point_estimate='mean',
                      rope_color='black')

In [ ]:
az.summary(idata)

In [ ]:
with basic_model:   
    prior_samples = pm.sample_prior_predictive(samples=1000, var_names=["alpha", "beta"])

In [ ]:
with basic_model:   
    y_test = pm.sample_posterior_predictive(idata, var_names=["alpha", "beta"])

In [ ]:
y_test

In [ ]:
idata.posterior.data_vars['alpha'].data.mean()

In [ ]:
prediction['Bayes'] = idata.posterior.data_vars['alpha'].data.mean() + idata.posterior.data_vars['beta'][0].data.mean()*data['x1']

In [ ]:
fig,ax=plt.subplots(sharex=True)
data_good.plot(x='x1', y='y_measure', style='.', ax=ax)
data_outliers.plot(x='x1', y='y_measure', label='outliers', style='.', ax=ax)
data.plot(x='x1', y='y', style='r-', ax=ax)

prediction.plot(x='x1', y='robust', style='--', ax=ax)
prediction.plot(x='x1', y='OLS', style='--', ax=ax)
prediction.plot(x='x1', y='Bayes', style='--', ax=ax)

In [ ]:
basic_model_robust = pm.Model()

with basic_model_robust:
    # Priors for unknown model parameters
    alpha = pm.Normal("alpha", mu=alpha_, sigma=0.1)
    beta = pm.Normal("beta", mu=beta_, sigma=0.1, shape=1)
    sigma = pm.HalfNormal("sigma", sigma=sigma_)

    # Expected value of outcome
    mu = alpha + beta[0] * data['x1']
    
    ν = pm.Uniform("ν", 1, 10)
    
    # Likelihood (sampling distribution) of observations
    # Using Student's T-distribution makes the regression more robust, (I don't know why...)
    Y_obs = pm.StudentT("Y_obs", nu=ν, mu=mu, sigma=sigma, observed=data['y_measure'])

In [ ]:
mu

In [ ]:
with basic_model_robust:
    # draw 1000 posterior samples
    trace_robust = pm.sample(draws=100)

In [ ]:
prediction['Bayes robust'] = trace_robust.posterior.data_vars['alpha'].data.mean() + trace_robust.posterior.data_vars['beta'][0].data.mean()*data['x1']

In [ ]:
fig,ax=plt.subplots(sharex=True)
data_good.plot(x='x1', y='y_measure', style='.', ax=ax)
data_outliers.plot(x='x1', y='y_measure', label='outliers', style='.', ax=ax)
data.plot(x='x1', y='y', style='r-', ax=ax)

prediction.plot(x='x1', y='robust', style='--', ax=ax)
prediction.plot(x='x1', y='OLS', style='--', ax=ax)
prediction.plot(x='x1', y='Bayes', style='--', ax=ax)
prediction.plot(x='x1', y='Bayes robust', style='--', ax=ax)

In [ ]:
az.plot_trace(trace_robust)